In [ ]:
pip install openpyxl pandas requests beautifulsoup4

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [3]:
df = pd.read_excel('registro_cobranca_inadimplentes.xlsx')

In [4]:
df.head()

,CODIGO_CLIENTE,NUMERO_CONTRATO,DATA_VENCIMENTO,DIA_VENCIMENTO,PARCELA_ATRASO,VL_ATRASO,EMPRESA_COBRANCA,DATA_PAGAMENTO,DIA_PAGAMENTO,VL_PAGAMENTO,VL_HONORARIOS_COBRANCA
0,8270,858591,2025-06-09,9,20,893.58,COBRADORA ABC LTDA,2025-06-16,16,893.58,116.17
1,1860,553407,2025-06-08,8,16,983.25,COBRADORA ABC LTDA,2025-06-16,16,983.25,127.82
2,6390,873746,2025-06-09,9,15,1290.22,COBRADORA ABC LTDA,2025-06-15,15,1290.22,167.73
3,6191,853658,2025-06-08,8,14,977.28,COBRADORA ABC LTDA,2025-06-18,18,977.28,127.05
4,6734,571849,2025-06-09,9,12,1283.28,COBRADORA ABC LTDA,2025-06-18,18,1283.28,166.83


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   CODIGO_CLIENTE          5000 non-null   int64         
 1   NUMERO_CONTRATO         5000 non-null   int64         
 2   DATA_VENCIMENTO         5000 non-null   datetime64[ns]
 3   DIA_VENCIMENTO          5000 non-null   int64         
 4   PARCELA_ATRASO          5000 non-null   int64         
 5   VL_ATRASO               5000 non-null   float64       
 6   EMPRESA_COBRANCA        5000 non-null   object        
 7   DATA_PAGAMENTO          5000 non-null   datetime64[ns]
 8   DIA_PAGAMENTO           5000 non-null   int64         
 9   VL_PAGAMENTO            5000 non-null   float64       
 10  VL_HONORARIOS_COBRANCA  5000 non-null   float64       
dtypes: datetime64[ns](2), float64(3), int64(5), object(1)
memory usage: 429.8+ KB


In [6]:
# Padronização das Colunas
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('ã', 'a')
    .str.replace('ç', 'c')
)


In [7]:
# Verificar duplicidades
duplicados = df.duplicated().sum()
print(f"Linhas duplicadas: {duplicados}")

Linhas duplicadas: 0


In [8]:
# Remover se houver
df = df.drop_duplicates()

In [ ]:
# Checar valores nulos
print(df.isna().sum())

# Caso queira eliminar linhas com valores nulos
# df = df.dropna()

# Ou preencher com zero/mediana se fizer sentido
# df['vl_honorarios_cobranca'].fillna(df['vl_honorarios_cobranca'].median(), inplace=True)
#

In [10]:
from datetime import datetime

# 1. Dias em atraso
df['dias_em_atraso'] = (df['data_pagamento'] - df['data_vencimento']).dt.days

# 2. Eficiência da cobrança (%)
df['eficiencia_cobranca'] = (df['vl_pagamento'] / df['vl_atraso']) * 100

# 3. Custo percentual da cobrança
df['custo_cobranca_percentual'] = (df['vl_honorarios_cobranca'] / df['vl_pagamento']) * 100

# 4. Flag de atraso (1 = houve atraso, 0 = não)
df['houve_atraso'] = df['dias_em_atraso'].apply(lambda x: 1 if x > 0 else 0)


In [11]:
# Verificar se há datas de pagamento antes do vencimento (inconsistência)
erros_datas = df[df['data_pagamento'] < df['data_vencimento']]
print(f"Pagamentos antes do vencimento: {len(erros_datas)}")

# Se quiser ajustar ou descartar:
# df = df[df['data_pagamento'] >= df['data_vencimento']]


Pagamentos antes do vencimento: 0


In [12]:
# Garantir formato correto
colunas_float = ['vl_atraso', 'vl_pagamento', 'vl_honorarios_cobranca']
df[colunas_float] = df[colunas_float].astype(float)


In [13]:
# Exportar para CSV
df.to_csv("dados_cobranca_limpo.csv", index=False)
